# Business Insights with SQL

SQL is the short for Structured Query Language. It is a domain-specific language used in programming and designed for managing data held in a relational database management systems. According to [Wikipedia](https://en.wikipedia.org/wiki/SQL), it is particularly useful in handling structured data, i.e. data incorporating relations among entities and variables.

In this project we will work with data from the [Chinook database](https://github.com/lerocha/chinook-database). The Chinook database contains information about a fictional digital music shop such as data about the artists, songs, playlists, music genres and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases.

Just as a curiosity, the database's name is inspired in another database, the Northwind database. According to its creator, the Chinook database is named after the winds in the interior West of North America, where the Canadian Prairies and Great Plains meet various mountain ranges. The Chinooks are most common over southern Alberta in Canada and that is why he thought it would be I good name for a database that intented to be an alternative to Northwind.

With all information that Chinook contains, there is an infinity of answers we could get out of this database. Therefore, the main goal of this project, is to write some queries to demonstrate some of the insights we can get from the data and how they could be applied to the real world. 

Although the code is available, this project does not aim to explain it in details. We'll, of course, explain what we are doing, what we expect form teh queries and what we can conclude from them. However, if you have any questions about the code itself [fell free to ask.](https://github.com/otavio-s-s/data_science/issues)

For the sake of simplicity, we'll assume that the fictional digital music shop is also named Chinook.


![](https://www.imagemhost.com.br/images/2020/05/12/3255539.jpg)
Image: <a href="https://www.freepik.com/free-photos-vectors/music">Music vector created by stories - www.freepik.com</a>

First, let's set up the SQL environment and then load the database.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## The Data

The Chinook database contains eleven tables and every table is connected to another. The following schema diagram can helps us to beter understand the table in this database, as well as their coulumns and conections to each other:

![](https://www.imagemhost.com.br/images/2020/05/12/Capturar.png)

In this schema diagram:

* The names in bold are the names of the tables;
* The columns with shading are the primary keys for each table;
* The connections between each table are shown by the black lines.

Now, let's write a query to see if these tables are actually in the database.

In [14]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ('table');

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


And we can see that the database contains all the tables in the diagram.

## The Queries

Now that we familiarized with the database, we can start querying it.


### Employees

In this first query, we are going to evaluate the work of our sales team. Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. So, this query returns the total dollar amount of sales assigned to each sales support agent in the company. We'll also be able to see for how long each of these employees have worked for Chinook and their average dollar amount of sales per month.

In [3]:
%%sql

WITH sales_suport_agents AS
    (
    SELECT
        e.*
    FROM employee e
    WHERE e.title = 'Sales Support Agent'
    )
    
SELECT
    ssa.first_name || ' ' || ssa.last_name agent_name,
    ROUND(SUM(i.total), 2) total_sold,
    CAST(julianday('now') -  julianday(ssa.hire_date) AS INTEGER) / 30 months_worked,
    ROUND(ROUND(SUM(i.total), 2) / (CAST(julianday('now') -  julianday(ssa.hire_date) AS INTEGER) / 30), 2) avg_sales_month
FROM sales_suport_agents ssa
INNER JOIN customer c ON ssa.employee_id = c.support_rep_id
INNER JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY 1
ORDER BY 4 DESC;

 * sqlite:///chinook.db
Done.


agent_name,total_sold,months_worked,avg_sales_month
Jane Peacock,1731.51,37,46.8
Steve Johnson,1393.92,31,44.97
Margaret Park,1584.0,36,44.0


With information like this we can easily evaluate the performance of Chinook's employees. For instance, although Margaret Park has a greater total dollar amount of sales assigned to her than Steve Johnson, we can see the Steve has been more productive since his average sales per month is higher. 

### Customers

Now we are going to look at the other side of sales and talk about customers.

The next query returns data on purchases from different countries. We will be able to see the total amount of sales as well as the average amount of dollar per customer and the average order value.

Notice that countries with only one costumer will be grouped as 'Other'.

In [4]:
%%sql

WITH countries_orders AS
    (
    SELECT
    CASE
        WHEN (
                SELECT COUNT(*) 
                FROM customer
                WHERE country = c.country
              ) = 1 THEN 'Other'
        ELSE c.country
        END AS country,
        c.customer_id,
        i.*
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    )

SELECT
    country,
    total_customers,
    total_sales,
    avg_value_per_costumer,
    avg_order_value
FROM
    (
    SELECT
        country,
        COUNT(DISTINCT customer_id) total_customers,
        ROUND(SUM(total), 2) total_sales,
        ROUND(SUM(total) / COUNT(DISTINCT customer_id), 2) avg_value_per_costumer,
        ROUND(SUM(total) / COUNT(DISTINCT invoice_id), 2) avg_order_value,
        CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
            END AS sort
    FROM countries_orders
    GROUP BY 1
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,avg_value_per_costumer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


Of course the sample size here is not good enough, but if we have more data this query can provide some useful insights. For instance, not only we can visualize which countries are already a established market for the company, but we can have an ideia of which countries have more potencial to become a successful market.  For example, we can consider as promising markets countries that, although do not present a large amount of total sales, present high average values per costumer and per order.

This kind of insights are surely helpful when a company needs to decide how much of its budget will be invested in each of the countries or even if one of them is not worthy any expenses at all.


### Platform Content and Consumer Preferences

We'll now investigate what kind of content the customers are more likely to purchase and what kind of ideas we can extract from this information.

The first query we'll write in this section returns the top 10 most popular music genres in the United States. The genres are sorted by the absolute number of tracks sold, but we can also see the percentage of the tracks sold he belong to each genre.

In [5]:
%%sql

WITH all_tracks_purchased AS
    (
    SELECT
        il.*
    FROM invoice_line il
    )
        
SELECT 
    g.name genre,
    COUNT(atp.invoice_line_id) tracks_sold,
    ROUND(CAST(COUNT(atp.invoice_line_id) AS FLOAT ) / ( SELECT COUNT(*) FROM all_tracks_purchased ), 2) percentage_sold
FROM all_tracks_purchased atp
INNER JOIN track t ON atp.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1 ORDER BY 2 DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,2635,0.55
Metal,619,0.13
Alternative & Punk,492,0.1
Latin,167,0.04
R&B/Soul,159,0.03
Blues,124,0.03
Jazz,121,0.03
Alternative,117,0.02
Easy Listening,74,0.02
Pop,63,0.01


In real life, this kind of information would be useful because if you know what your audience prefers, then you can add  more songs that belong to that genre to the platform, or even increase the advertisement on the genres that are not doing so well

Another very useful information is the most purchased artists. That's what the next query will return.

In [6]:
%%sql

SELECT
    a.name,
    COUNT(il.track_id) number_of_tracks
FROM artist a
INNER JOIN album al ON a.artist_id = al.artist_id
INNER JOIN track t ON al.album_id = t.album_id
INNER JOIN invoice_line il ON t.track_id = il.track_id
GROUP BY 1 ORDER BY 2 DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


name,number_of_tracks
Queen,192
Jimi Hendrix,187
Red Hot Chili Peppers,130
Nirvana,130
Pearl Jam,129
Guns N' Roses,124
AC/DC,124
Foo Fighters,121
The Rolling Stones,117
Metallica,106


With the last two queries we can have a pretty good ideia of what kind of music the customers like most and use such information to define the company's marketing and tracks purchasing strategies, for example.

We also can see how the number change in different locations in order to better adapt the estrategies to each country o even to each state.

Let's see this same information, but now only for the customers living in the United States.

In [7]:
%%sql

WITH usa_tracks AS
    (
    SELECT
        il.*
    FROM invoice_line il
    INNER JOIN invoice i ON il.invoice_id = i.invoice_id
    INNER JOIN customer c ON  i.customer_id = c.customer_id
    WHERE c.country = 'USA'
    )
        
SELECT 
    g.name genre,
    COUNT(ut.invoice_line_id) tracks_sold,
    ROUND(CAST(COUNT(ut.invoice_line_id) AS FLOAT ) / ( SELECT COUNT(*) FROM usa_tracks ), 2) percentage_sold
FROM usa_tracks ut
INNER JOIN track t ON ut.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1 ORDER BY 2 DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Pop,22,0.02
Latin,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


In [8]:
%%sql

SELECT
    a.name,
    COUNT(il.track_id) number_of_tracks
FROM artist a
INNER JOIN album al ON a.artist_id = al.artist_id
INNER JOIN track t ON al.album_id = t.album_id
INNER JOIN invoice_line il ON t.track_id = il.track_id
INNER JOIN invoice i ON il.invoice_id = i.invoice_id
INNER JOIN customer c ON i.customer_id = c.customer_id
WHERE c.country = 'USA'
GROUP BY 1 ORDER BY 2 DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


name,number_of_tracks
Van Halen,43
R.E.M.,38
The Rolling Stones,37
Nirvana,35
Foo Fighters,34
Eric Clapton,34
Guns N' Roses,32
Green Day,32
Pearl Jam,31
Amy Winehouse,30


We can see some small differences in the genre preferences and some considerable differences on artist preferences. As said, this can be used to optimize the company's strategies for each location.

As we're talking about strategies, let's see if the range of tracks in the store is reflective of their sales popularity. As we saw earlier, the top 10 most purchased genres are:

|genre	           | percentage_sold|
|------------------|----------------|
|Rock	           |   	        0.55|
|Metal	           |            0.13|
|Alternative & Punk|   	        0.10|
|Latin	           |            0.04|
|R&B/Soul	       |   	        0.03|
|Blues	           |  	        0.03|
|Jazz	           |  	        0.03|
|Alternative       |  	        0.02|
|Easy Listening	   |            0.02|
|Pop	           |            0.01|

Now, we'll write a query to see top 10 most common genres in the store and see if it matches the top 10 most purchased genres.

In [9]:
%%sql
WITH all_tracks AS
    (
    SELECT
        t.*
    FROM track t
    )
    

SELECT
    g.name genre,
    ROUND(CAST(COUNT(t.track_id) AS FLOAT) / (SELECT COUNT(*) FROM all_tracks ), 2) percentage_tracks
FROM all_tracks at
INNER JOIN track t ON at.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1 ORDER BY 2 DESC
LIMIT 10



 * sqlite:///chinook.db
Done.


genre,percentage_tracks
Rock,0.37
Latin,0.17
Metal,0.11
Alternative & Punk,0.09
Jazz,0.04
TV Shows,0.03
Reggae,0.02
R&B/Soul,0.02
Drama,0.02
Classical,0.02


One insight we can take from this is abou the Latin genre. While this genre represents 17% of the musics available in the platform, it only represents 4% of the purchased tracks. With this information, Chinook can take some action to fix this. They could either promote the Latin music on the platform, or make the platform more friendly to Latin people (maybe with a Spanish version) or even reduce the amount of Latin music it buys from the recorders.

Let's keep working with the customers preferences. We'll investigate a little on the customers habits when making a purchase.

The following query returns the percentage of purchases that are individual tracks and the percentage that are whole albums. For this job, we will identify whether each invoice has all the tracks from an album. We can accomplish this by getting all the of tracks from an invoice and comparing it to the tracks from an album.

In [10]:
%%sql

WITH invoice_first_track AS
    (
    SELECT
        il.invoice_id invoice_id,
        MIN(il.track_id) first_track_id
    FROM invoice_line il
    GROUP BY 1
    )
    
SELECT 
    album_purchase,
    COUNT(invoice_id) invoices,
    CAST(COUNT(invoice_id) AS FLOAT) / (
                                        SELECT COUNT (*) FROM invoice
                                    ) percent
FROM
    (
    SELECT 
        ift.*,
        CASE
            WHEN
                (
                SELECT t.track_id FROM track t
                WHERE t.album_id = (
                                    SELECT t2.album_id FROM track t2
                                    WHERE t2.track_id = ift.first_track_id
                                    )
                    
                EXCEPT
                
                SELECT il2.track_id FROM invoice_line il2
                WHERE il2.invoice_id = ift.invoice_id
                ) IS NULL
            AND
                (
                SELECT il2.track_id FROM invoice_line il2
                WHERE il2.invoice_id = ift.invoice_id
                    
                EXCEPT
                    
                SELECT t.track_id FROM track t
                WHERE t.album_id = (
                                    SELECT t2.album_id FROM track t2
                                    WHERE t2.track_id = ift.first_track_id
                                    )
                ) IS NULL
            THEN 'yes'
            ELSE 'no'
            END AS 'album_purchase'
        FROM invoice_first_track ift
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


We can see that a substancial part of the invoices correspond to entire albums, almost twenty per cent of them. If that was not the case, then maybe it would be worth to investigate if there's any songs that are never purchased and use this information to rethink the way Chinook buys the tracks from the recorders. Instead of buying full album, they could start to think about buying only the kind os song that are more likely to be purchased by the Chinook's customers.

As we approach the end of this project, we'll see how Chinook is handling the protected media. 

In the `media_type` table we can see the tracks are divided into five different types of media, as you can see with the query below.

In [11]:
%%sql
SELECT DISTINCT * FROM media_type

 * sqlite:///chinook.db
Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


What we want to check here is if the amount of protected media in the platform reflects the amount of protected media purchased by the costumers. First, let's see how many protected an unprotected files we have in the platform.

In [12]:
%%sql

WITH all_tracks AS
    (
    SELECT
        t.*
    FROM track t
    )
    
SELECT
    CASE
        WHEN mt.name LIKE "Protected%"
        THEN 'yes'
        ELSE 'no'
        END AS protected,
    COUNT(t.name) tracks_available,
    ROUND(CAST(COUNT(t.name) AS FLOAT) / (SELECT COUNT(*) FROM all_tracks ), 2) percentage_available
FROM all_tracks at
INNER JOIN track t ON at.track_id = t.track_id
INNER JOIN media_type mt ON t.media_type_id = mt.media_type_id
GROUP BY 1

 * sqlite:///chinook.db
Done.


protected,tracks_available,percentage_available
no,3052,0.87
yes,451,0.13


And now how many protected an unprotected tracks were purchased.

In [13]:
%%sql

WITH all_tracks_purchased AS
    (
    SELECT
        il.*
    FROM invoice_line il
    )

SELECT
    CASE
        WHEN mt.name LIKE "Protected%"
        THEN 'yes'
        ELSE 'no'
        END AS protected,
    COUNT(il.track_id) tracks_purchased,
    ROUND(CAST(COUNT(il.track_id) AS FLOAT) / (SELECT COUNT(*) FROM all_tracks_purchased), 2) percentage_purchased
FROM all_tracks_purchased atp
INNER JOIN invoice_line il ON atp.invoice_line_id = il.invoice_line_id
INNER JOIN track t ON il.track_id = t.track_id
INNER JOIN media_type mt ON t.media_type_id = mt.media_type_id
GROUP BY 1


 * sqlite:///chinook.db
Done.


protected,tracks_purchased,percentage_purchased
no,4315,0.91
yes,442,0.09


We can see that the numbers are pretty close here, so we can say that the amount of protected files available in the platform makes perfect sense.

## Conclusion

As we reach the end of this project, we are now able to conclude the SQL is a very powerful solution to investigate larges amounts of data spread into different tables and extract important insights from it.

Thank you for reading this and if you have any questions, suggestions of feedback feel free to contact me.